In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
import zipfile
import tempfile
import torch
from PIL import Image
from tqdm import tqdm
import random
from transformers import BlipProcessor, BlipForConditionalGeneration

# Define paths
scienceqa_path = "./ScienceQA_DATA"
output_path = "./ScienceQA_DATA/processed_output"
os.makedirs(os.path.join(output_path, "images"), exist_ok=True)

# Load problems data
with open(os.path.join(scienceqa_path, "problems.json"), "r") as f:
    problems = json.load(f)

# Load BLIP model for captioning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
blip_model.eval()

# Function to generate caption using BLIP
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# Load BLIP-2 model (more advanced than BLIP)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device)

def generate_caption(image, context_text):
    # Check if context_text is a string or a dictionary
    if isinstance(context_text, dict):
        # If it's a dictionary (problem data), extract relevant information
        question = context_text.get("question", "")
        choices = context_text.get("choices", [])
        answer_idx = context_text.get("answer", -1)
        answer = choices[answer_idx] if 0 <= answer_idx < len(choices) else ""
        topic = context_text.get("topic", "")
        subject = context_text.get("subject", "")

        # Create an instructive prompt
        prompt = f"Describe this {subject} diagram about {topic} that relates to: {question}"
    else:
        # If it's already a string, use it directly
        prompt = context_text

    # Process with BLIP
    inputs = blip_processor(images=image, text=prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = blip_model.generate(**inputs)
        caption = blip_processor.decode(out[0], skip_special_tokens=True)

    return caption

# Function to extract and process images
def extract_and_process_images(zip_path, split_name, target_count=3000):
    processed_count = 0
    processed_data = []

    # Create a temporary directory to extract files
    with tempfile.TemporaryDirectory() as temp_dir:
        print(f"Extracting {zip_path} to {temp_dir}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        split_dir = os.path.join(temp_dir, split_name)
        if not os.path.exists(split_dir):
            print(f"Warning: Expected directory {split_dir} not found")
            split_dir = temp_dir

        print(f"Looking for question folders in {split_dir}")

        def is_scientific_domain(problem):
            subject = problem.get("subject", "").lower()
            topic = problem.get("topic", "").lower()
            if "natural" in subject:
                return True
            if "social" in subject and ("geography" in topic or "map" in topic):
                return True
            return False

        def is_likely_diagram(problem):
            text = (problem.get("question", "") + " " +
                    " ".join(problem.get("choices", [])) + " " +
                    problem.get("lecture", "")).lower()
            diagram_keywords = ["diagram", "figure", "chart", "graph", "labeled",
                                "structure", "system", "cycle", "process", "anatomy"]
            return any(keyword in text for keyword in diagram_keywords)

        domain_counts = {
            "biology": 0, "physics": 0, "chemistry": 0,
            "earth_science": 0, "geography": 0,
            "natural_science": 0, "other": 0
        }
        domain_targets = {
            "biology": 3000, "physics": 2000, "chemistry": 1500,
            "earth_science": 1500, "geography": 500,
            "natural_science": 1000, "other": 500
        }

        priority_folders = []
        regular_folders = []

        for folder_name in os.listdir(split_dir):
            folder_path = os.path.join(split_dir, folder_name)
            if not os.path.isdir(folder_path):
                continue
            if folder_name in problems and problems[folder_name].get("image") == "image.png":
                problem = problems[folder_name]
                if is_scientific_domain(problem) and is_likely_diagram(problem):
                    priority_folders.append(folder_name)
                else:
                    regular_folders.append(folder_name)

        all_folders = priority_folders + regular_folders
        print(f"Found {len(priority_folders)} priority folders and {len(regular_folders)} regular folders")

        for folder_name in tqdm(all_folders):
            folder_path = os.path.join(split_dir, folder_name)
            if processed_count >= target_count:
                break
            if folder_name not in problems:
                continue
            problem = problems[folder_name]
            if problem.get("image") != "image.png" or problem.get("split") != split_name:
                continue
            img_path = os.path.join(folder_path, "image.png")
            if not os.path.exists(img_path):
                continue

            subject = problem.get("subject", "").lower()
            topic = problem.get("topic", "").lower()

            if "natural" in subject:
                if "biology" in topic or "life" in topic:
                    domain = "biology"
                elif "physics" in topic:
                    domain = "physics"
                elif "chemistry" in topic:
                    domain = "chemistry"
                elif "earth" in topic or "space" in topic:
                    domain = "earth_science"
                else:
                    domain = "natural_science"
            elif "social" in subject and ("geography" in topic or "map" in topic):
                domain = "geography"
            else:
                domain = "other"

            if domain_counts[domain] >= domain_targets.get(domain, 500):
                continue

            unique_id = f"scienceqa_{folder_name}"
            dest_img_path = os.path.join(output_path, "images", f"{unique_id}.png")

            try:
                img = Image.open(img_path).convert("RGB")
                if img.width < 200 or img.height < 200:
                    continue
                img.thumbnail((512, 512))
                new_img = Image.new("RGB", (512, 512), (255, 255, 255))
                paste_pos = ((512 - img.width) // 2, (512 - img.height) // 2)
                new_img.paste(img, paste_pos)
                new_img.save(dest_img_path)

                question = problem.get("question", "")
                choices = problem.get("choices", [])
                lecture = problem.get("lecture", "")
                context_text = question + " " + " ".join(choices) + " " + lecture

                caption = generate_caption(new_img, problem)

                entry = {
                    "image_id": unique_id,
                    "image_path": f"images/{unique_id}.png",
                    "caption": caption,
                    "domain": domain,
                    "subject": problem.get("subject", ""),
                    "topic": problem.get("topic", ""),
                    "source": "scienceqa"
                }

                processed_data.append(entry)
                domain_counts[domain] += 1
                processed_count += 1

                if processed_count % 100 == 0:
                    print(f"Processed {processed_count} images")

            except Exception as e:
                print(f"Error processing {img_path}: {e}")

    print("Domain distribution:")
    for domain, count in domain_counts.items():
        print(f"  {domain}: {count}")

    return processed_data

# Process each split
all_data = []

train_zip = os.path.join(scienceqa_path, "train.zip")
if os.path.exists(train_zip):
    train_data = extract_and_process_images(train_zip, "train", target_count=4500)
    all_data.extend(train_data)
    print(f"Processed {len(train_data)} images from train")

val_zip = os.path.join(scienceqa_path, "val.zip")
if os.path.exists(val_zip):
    val_data = extract_and_process_images(val_zip, "val", target_count=2500)
    all_data.extend(val_data)
    print(f"Processed {len(val_data)} images from val")

test_zip = os.path.join(scienceqa_path, "test.zip")
if os.path.exists(test_zip):
    test_data = extract_and_process_images(test_zip, "test", target_count=2000)
    all_data.extend(test_data)
    print(f"Processed {len(test_data)} images from test")

random.shuffle(all_data)

total = len(all_data)
if total > 0:
    train_size = int(total * 0.8)
    val_size = int(total * 0.1)

    train_data = all_data[:train_size]
    val_data = all_data[train_size:train_size + val_size]
    test_data = all_data[train_size + val_size:]

    for split_name, split_data in [
        ("train", train_data),
        ("val", val_data),
        ("test", test_data)
    ]:
        with open(os.path.join(output_path, f"{split_name}.jsonl"), "w") as f:
            for entry in split_data:
                f.write(json.dumps(entry) + "\n")
        print(f"Saved {len(split_data)} examples to {split_name}.jsonl")

    with open(os.path.join(output_path, "metadata.jsonl"), "w") as f:
        for entry in all_data:
            f.write(json.dumps(entry) + "\n")
    print(f"Saved {len(all_data)} total examples to metadata.jsonl")
else:
    print("No images were processed!")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.57it/s]


Extracting ./ScienceQA_DATA/train.zip to /tmp/tmpqsfqyom4...
Looking for question folders in /tmp/tmpqsfqyom4/train
Found 1307 priority folders and 4911 regular folders


  2%|▏         | 131/6218 [00:05<03:34, 28.36it/s]

Processed 100 images


  4%|▍         | 259/6218 [00:09<03:16, 30.38it/s]

...

Processed 1200 images


 88%|████████▊ | 1774/2017 [00:58<00:02, 91.76it/s]

Processed 1300 images


100%|██████████| 2017/2017 [01:02<00:00, 32.08it/s]


Domain distribution:
  biology: 265
  physics: 135
  chemistry: 85
  earth_science: 122
  geography: 500
  natural_science: 129
  other: 156
Processed 1392 images from test
Saved 4941 examples to train.jsonl
Saved 617 examples to val.jsonl
Saved 619 examples to test.jsonl
Saved 6177 total examples to metadata.jsonl


In [24]:
!pip install pytorch_lightning

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.8/219.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.0/334.0 kB 31.1 MB/s eta 0:00:00


In [ ]:
import os
import json
import torch
import time
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, LlavaForConditionalGeneration
import multiprocessing
from functools import partial
import concurrent.futures

# === Paths ===
image_dir = "ScienceQA_DATA/processed_output/images"
metadata_in = "ScienceQA_DATA/processed_output/metadata.jsonl"
metadata_out = "ScienceQA_DATA/processed_output/image_captions.jsonl"

# === Optimization settings ===
# Image preprocessing in parallel
NUM_LOADER_WORKERS = 4
# Number of images to load at once
PREFETCH_FACTOR = 2
# Precompute image paths for faster access
USE_IMAGE_CACHE = True

# === Helper functions ===
def preprocess_image(image_path):
    """Load and preprocess image - this can be done in parallel"""
    try:
        image = Image.open(image_path).convert("RGB")
        # Resize to model input size to reduce memory usage and computation
        image = image.resize((224, 224))
        return image
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

def prepare_dataset(metadata_file, image_dir, use_cache=True):
    """Prepare dataset with image paths"""
    # Check if cached file exists
    cache_file = os.path.join(os.path.dirname(metadata_file), "valid_images_cache.json")

    if use_cache and os.path.exists(cache_file):
        print(f"Loading cached dataset from {cache_file}")
        with open(cache_file, 'r') as f:
            return json.load(f)

    print("Scanning dataset and validating images...")
    valid_entries = []

    # Read all entries
    all_entries = []
    with open(metadata_file, 'r') as f:
        for line in f:
            try:
                entry = json.loads(line.strip())
                all_entries.append(entry)
            except:
                pass

    # Validate images in parallel to speed up preparation
    image_paths = [os.path.join(image_dir, os.path.basename(entry.get("image_path", "")))
                  for entry in all_entries]

    # Use multiple processes to check file existence
    with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_LOADER_WORKERS) as executor:
        exists_results = list(tqdm(
            executor.map(os.path.exists, image_paths),
            total=len(image_paths),
            desc="Validating images"
        ))

    # Create final dataset with valid entries
    for entry, image_path, exists in zip(all_entries, image_paths, exists_results):
        if exists:
            image_id = entry.get('image_id', os.path.basename(image_path))
            valid_entries.append({
                "entry": entry,
                "image_path": image_path,
                "image_id": image_id
            })

    # Cache the results for future runs
    if use_cache:
        with open(cache_file, 'w') as f:
            json.dump(valid_entries, f)
        print(f"Cached {len(valid_entries)} valid entries to {cache_file}")

    print(f"Found {len(valid_entries)} valid images to process")
    return valid_entries

# === Caption Generator ===
class OptimizedCaptionGenerator:
    def __init__(self, model_id="llava-hf/llava-1.5-7b-hf"):
        self.model_id = model_id

        # Load model with optimizations
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Loading model on {self.device}...")

        self.processor = AutoProcessor.from_pretrained(model_id)

        # Load model with optimizations
        dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        self.model = LlavaForConditionalGeneration.from_pretrained(
            model_id,
            torch_dtype=dtype
        ).to(self.device)

        # Set to evaluation mode
        self.model.eval()

        # Optimize torch operations
        torch.set_grad_enabled(False)
        if torch.cuda.is_available():
            torch.backends.cudnn.benchmark = True

    def generate_captions_batch(self, items):
        """Generate captions for a batch of images"""
        results = []

        # Pre-process all images in parallel
        image_paths = [item["image_path"] for item in items]

        # Process images in parallel using a thread pool
        with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_LOADER_WORKERS) as executor:
            images = list(executor.map(preprocess_image, image_paths))

        # For each valid image, generate caption
        for item, image in zip(items, images):
            if image is None:
                continue

            image_id = item["image_id"]
            try:
                # Simple prompt for scientific images
                prompt = "<image>\nDescribe this diagram briefly."

                # Process input
                inputs = self.processor(text=prompt, images=image, return_tensors="pt").to(self.device)

                # Generate with optimized parameters
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=50,
                    do_sample=False,  # Use greedy decoding for speed
                    num_beams=1,      # No beam search for speed
                    repetition_penalty=1.2
                )

                # Decode caption
                caption = self.processor.decode(outputs[0], skip_special_tokens=True).strip()

                # Clean up caption - simplify for speed
                if "Describe this" in caption:
                    caption = caption.replace("Describe this", "").strip()
                if "diagram briefly" in caption:
                    caption = caption.replace("diagram briefly", "").strip()

                caption = caption.lstrip(" ,.:-")

                # Format correctly (but minimal processing for speed)
                if caption and len(caption) > 0:
                    caption = caption[0].upper() + caption[1:]
                if not caption.endswith(('.', '!', '?')):
                    caption += '.'

                # Add to results
                results.append({
                    "image": image_id,
                    "caption": caption
                })

            except Exception as e:
                print(f"Error generating caption for {image_id}: {e}")

        return results

# === Main processing function ===
def process_dataset(batch_size=8, checkpoint_every=100, start_from=0):
    # Prepare dataset
    dataset = prepare_dataset(metadata_in, image_dir, use_cache=USE_IMAGE_CACHE)

    if len(dataset) == 0:
        print("No valid images found. Check paths and metadata.")
        return

    # Initialize generator
    generator = OptimizedCaptionGenerator()

    # Load existing results if continuing
    all_results = []
    if start_from > 0 and os.path.exists(metadata_out):
        with open(metadata_out, 'r') as f:
            for line in f:
                all_results.append(json.loads(line))
        print(f"Loaded {len(all_results)} existing results")

    # Process in batches
    total_batches = (len(dataset) - start_from + batch_size - 1) // batch_size

    print(f"Processing {len(dataset) - start_from} images in {total_batches} batches...")
    start_time = time.time()

    for i in tqdm(range(start_from, len(dataset), batch_size), desc="Processing batches"):
        batch = dataset[i:min(i + batch_size, len(dataset))]
        batch_start = time.time()

        # Generate captions
        batch_results = generator.generate_captions_batch(batch)
        all_results.extend(batch_results)

        # Print sample and timing
        if batch_results:
            batch_time = time.time() - batch_start
            images_per_second = len(batch_results) / batch_time

            print(f"\nSample result - Image: {batch_results[0]['image']}")
            print(f"Caption: {batch_results[0]['caption']}")
            print(f"Batch processed in {batch_time:.2f}s ({images_per_second:.2f} images/sec)")

        # Save checkpoint periodically
        if (i - start_from) % checkpoint_every == 0 or i + batch_size >= len(dataset):
            with open(metadata_out, "w") as outfile:
                for result in all_results:
                    outfile.write(json.dumps(result) + "\n")

            # Calculate overall speed
            elapsed = time.time() - start_time
            total_processed = min(i + batch_size, len(dataset)) - start_from
            overall_speed = total_processed / elapsed if elapsed > 0 else 0

            print(f"Saved checkpoint with {len(all_results)} results")
            print(f"Overall speed: {overall_speed:.2f} images/sec, ETA: {(len(dataset)-i-batch_size)/overall_speed/3600:.1f} hours")

    # Final stats
    total_time = time.time() - start_time
    print(f"\nProcessed {len(all_results)} images in {total_time:.2f} seconds")
    print(f"Average speed: {len(all_results)/total_time:.2f} images/second")
    print(f"Captions saved to {metadata_out}")

if __name__ == "__main__":
    # Set parameters - TUNED FOR SPEED
    BATCH_SIZE = 8         # Larger batches for efficiency
    CHECKPOINT_EVERY = 100  # Save periodically
    START_FROM = 0          # Start position

    # Run the process
    process_dataset(batch_size=BATCH_SIZE, checkpoint_every=CHECKPOINT_EVERY, start_from=START_FROM)

Scanning dataset and validating images...


Validating images: 100%|██████████| 6177/6177 [00:01<00:00, 5489.00it/s]


Cached 6177 valid entries to ScienceQA_DATA/processed_output/valid_images_cache.json
Found 6177 valid images to process
Loading model on cuda...


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 25.87it/s]


Processing 6177 images in 773 batches...


...


Sample result - Image: scienceqa_12888
Caption: The image is a large, green map of the United States with various states and territories labeled on it. There are also several smaller maps within the larger one that showcase different regions or areas in more detail.
Batch processed in 222.38s (0.04 images/sec)


Processing batches:  89%|████████▉ | 687/773 [40:34:22<5:10:07, 216.37s/it]


Sample result - Image: scienceqa_3617
Caption: The image shows a purple molecule with six atoms, each connected to the central atom by a bond. These bonds are represented as lines connecting the atoms in various directions. There is also an additional small circle located near one of these b.
Batch processed in 222.43s (0.04 images/sec)


Processing batches:  89%|████████▉ | 688/773 [40:38:30<5:19:45, 225.71s/it]


Sample result - Image: scienceqa_6416
Caption: The image shows a solar panel on top of an antenna, which is mounted to the side of a hill or mountain. This setup appears to be used for monitoring weather conditions and possibly providing power through solar energy.
Batch processed in 247.50s (0.03 images/sec)
